In [1]:
import textract

In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
#extraigo texto
archivo= 'tresalia\Bancos\Banamex_nacional\pdf\cuenta_cheques_banamex PDF.pdf'

In [4]:
text = textract.process(archivo)

In [5]:
#cambio formato
encoding = 'utf-8'
text_str=text.decode(encoding)

In [6]:
#separo
pdf=re.split(r"[~\r\n]+", text_str)

In [7]:
information_pdf=pd.DataFrame(pdf).rename(columns={0:"informacion"})
information_pdf.head()

,informacion
0,cffibanamex@
1,ESTADO DE CUENTA AL 30 DE SEPTIEMBRE DE 2020
2,CLIENTE:
3,9999999
4,Registro F ederal de Contribuyentes:


In [8]:
#cliente
cliente_lista=[]
for i in range(len(pdf)):
    if pdf[i].lower().startswith('cliente'):
        cliente_lista.append(pdf[i])
        cliente_lista.append(pdf[i+1])
        
cliente_lista

['CLIENTE:', '9999999', 'CLIENTE:', '9999999', 'CLIENTE:', '9999999']

In [9]:
cliente= 'cliente '+cliente_lista[1]
cliente

'cliente 9999999'

In [10]:
#nombre del cliente
nombre_lista=[]
for i in pdf:
    if i.startswith("ooo"):
        nombre_lista.append(i)
        
nombre_lista

['ooo o "\' NOMBRE APELLIDO P APELLIDO M']

In [11]:
nombre=nombre_lista[0].split(' ')

In [12]:
nombre_cliente=[]
for i in nombre:
    if i.isupper():
        nombre_cliente.append(i)

In [13]:
nombre=' '.join(nombre_cliente)
nombre

'NOMBRE APELLIDO P APELLIDO M'

In [14]:
#periodo
periodo_lista=[]
for i in pdf:
    if i.startswith("RESUMEN DEL"):
        periodo_lista.append(i)
        
periodo_lista

['RESUMEN DEL: 01/SEP/2020 AL 30/SEP/2020']

In [15]:
periodo= periodo_lista[0]
periodo

'RESUMEN DEL: 01/SEP/2020 AL 30/SEP/2020'

In [16]:
#rfc
rfc_lista=[]
for i in range(len(pdf)):
    if re.search('contribuyentes', pdf[i].lower()):
        rfc_lista.append(pdf[i])
        rfc_lista.append(pdf[i+1])
        
rfc_lista

['Registro F ederal de Contribuyentes:', 'XXXX999999XX9']

In [17]:
rfc='RFC '+ rfc_lista[1]
rfc

'RFC XXXX999999XX9'

In [18]:
#localizo la tabla a partir de la palabra "detalle operaciones"
indices_detalle=[]
for i in range(len(pdf)):
    if re.search('detalle de operaciones',pdf[i].lower()):
        indices_detalle.append(i)

In [19]:
indices_detalle

[46, 77]

In [20]:
# en la automatización se puede hacer un for mas grande con los indeces para casos de muchas páginas
for i in range(len(indices_detalle)):
    print(i)

0
1


# todas las páginas juntas

In [28]:
tabla_pag_1= pdf[indices_detalle[0]:-1]
tabla_pag_1

['DETALLE DE OPERACIONES',
 'DEPOSITOS $9,984.00',
 'FECHA 01 SEP',
 'CONCEPTO SALDO ANTERIOR PAGO RECIBIDO DE BMONEX POR ORDEN DE NOMBRE XXXXXXXXXXX CTA.ORDENANTE 000000000000000000 REF.0000000 TRASPASO A CUENTA PROPIA RASTREO: 00000000',
 '01 SEP 02 SEP 10 SEP 17 SEP 17 SEP',
 'COBRO DE CHEQUE NUMERO 9999 COBRO DE CHEQUE NUMERO 9999 COBRO DE CHEQUE NUMERO 9999 COBRO DE CHEQUE NUMERO 9999 PAGO RECIBIDO DE BMONEX POR',
 'RETIROS',
 'DEPOSITOS',
 'SALDO',
 '1',
 '9,021.42',
 '1',
 '1',
 '1-',
 ',_',
 ',-',
 '5,000.00',
 ',_ 3,000.00',
 '9,021.42',
 ',- 3,000.00',
 ',- 9,021.42',
 ', 3,000.00 3,000.00',
 ',_ 9,021.42',
 '000184.B08CHDA001.OD.0930.01',
 '\x0ccffibanamex@ ESTADO DE CUENTA AL 30 DE SEPTIEMBRE DE 2020',
 'CLIENTE:',
 '9999999',
 'Página:',
 '2 de7',
 'NOMBRE APELLIDO P APELLIDO M',
 '7',
 'DETALLE DE OPERACIONES',
 'FECHA',
 'CONCEPTO ORDEN DE XXXXX XXXXXXXXXXXXXX XXXXXXX CTA.ORDENANTE 000000000000000000 REF.0000000 TRASPASO A CUENTA PROPIA RASTREO: 00000000',
 '18 SEP 24 SE

In [29]:
#fechas
fecha_lista_pag_1=[]
for i in range(len(tabla_pag_1)):
    if re.match('fecha',tabla_pag_1[i].lower()):
        fecha_lista_pag_1.append(tabla_pag_1[i])
    elif re.match(r'\d{2} \w{3}',tabla_pag_1[i]):
        fecha_lista_pag_1.append(tabla_pag_1[i])

        
fecha_lista_pag_1

['FECHA 01 SEP',
 '01 SEP 02 SEP 10 SEP 17 SEP 17 SEP',
 'FECHA',
 '18 SEP 24 SEP',
 '24 SEP 24 SEP 24 SEP 30 SEP',
 '30 SEP COBRO DE CHEQUE NUMERO 0000 30 SEP NUMERO DE CHEQUES EXENTOS']

In [30]:
#separo
fechas_lista_2_pag_1=[]

for i in fecha_lista_pag_1:
    fechas_lista_2_pag_1.append(re.findall(r'\d{2} \w{3}',i))

In [31]:
fechas_lista_2_pag_1

[['01 SEP'],
 ['01 SEP', '02 SEP', '10 SEP', '17 SEP', '17 SEP'],
 [],
 ['18 SEP', '24 SEP'],
 ['24 SEP', '24 SEP', '24 SEP', '30 SEP'],
 ['30 SEP', '30 SEP']]

In [74]:
#aplano lista de listas
fechas_pag_1 = ['0']
for i in fechas_lista_2_pag_1:
    for j in i:
        fechas_pag_1.append(j)

In [75]:
fechas_pag_1

['0',
 '01 SEP',
 '01 SEP',
 '02 SEP',
 '10 SEP',
 '17 SEP',
 '17 SEP',
 '18 SEP',
 '24 SEP',
 '24 SEP',
 '24 SEP',
 '24 SEP',
 '30 SEP',
 '30 SEP',
 '30 SEP']

In [68]:
#conceptos
conceptos_lista_pag_1=[]
for i in range(len(tabla_pag_1)):
    if re.match('concepto',tabla_pag_1[i].lower()):
        conceptos_lista_pag_1.append(tabla_pag_1[i])
        
    elif re.match('cobro',tabla_pag_1[i].lower()):
        conceptos_lista_pag_1.append(tabla_pag_1[i])
        
    elif re.match('pago',tabla_pag_1[i].lower()):
        conceptos_lista_pag_1.append(tabla_pag_1[i])
        
    elif re.match('expedicion',tabla_pag_1[i].lower()):
        conceptos_lista_pag_1.append(tabla_pag_1[i])
        
    elif re.match('por instruccion',tabla_pag_1[i].lower()):
        conceptos_lista_pag_1.append(tabla_pag_1[i])
        
    elif re.search('exentos',tabla_pag_1[i].lower()):
        conceptos_lista_pag_1.append(tabla_pag_1[i])
        conceptos_lista_pag_1.append(tabla_pag_1[i+1])
       
        
conceptos_lista_pag_1

['CONCEPTO SALDO ANTERIOR PAGO RECIBIDO DE BMONEX POR ORDEN DE NOMBRE XXXXXXXXXXX CTA.ORDENANTE 000000000000000000 REF.0000000 TRASPASO A CUENTA PROPIA RASTREO: 00000000',
 'COBRO DE CHEQUE NUMERO 9999 COBRO DE CHEQUE NUMERO 9999 COBRO DE CHEQUE NUMERO 9999 COBRO DE CHEQUE NUMERO 9999 PAGO RECIBIDO DE BMONEX POR',
 'CONCEPTO ORDEN DE XXXXX XXXXXXXXXXXXXX XXXXXXX CTA.ORDENANTE 000000000000000000 REF.0000000 TRASPASO A CUENTA PROPIA RASTREO: 00000000',
 'COBRO DE CHEQUE NUMERO 9999 PAGO RECIBIDO DE BMONEX POR ORDEN DE XXXXXXX XXXXXXXXXXXXXX XXXXXX CTA.ORDENANTE 000000000000000000 REF.0000000 TRASPASO A CUENTA PROPIA RASTREO: 00000000',
 'EXPEDICION ORDEN AL EXTRANJERO 0000000 O.P. ENVIADA AL EXT. POR INSTRUCCION DEL CUENTE 0000000 O.P. ENVIADA AL EXT. POR INSTRUCCION DEL CLIENTE 0000000 O.P. ENVIADA AL EXT. PAGO RECIBIDO DE BMONEX POR ORDEN DE XXXXXXXX XXXXXXXXXXXXXX XXXXXX CTA.ORDENANTE 000000000000000000 REF.0000000 TRASPASO A CUENTA PROPIA RASTREO: 00000000',
 '30 SEP COBRO DE CHEQUE 

In [69]:
conceptos1_1=' '.join(conceptos_lista_pag_1)
conceptos1_2=re.split('SALDO |PAGO | COB|POR INSTRUCCION |EXPE|NUMERO DE CHE', conceptos1_1)
conceptos1_2

['CONCEPTO ',
 'ANTERIOR ',
 'RECIBIDO DE BMONEX POR ORDEN DE NOMBRE XXXXXXXXXXX CTA.ORDENANTE 000000000000000000 REF.0000000 TRASPASO A CUENTA PROPIA RASTREO: 00000000',
 'RO DE CHEQUE NUMERO 9999',
 'RO DE CHEQUE NUMERO 9999',
 'RO DE CHEQUE NUMERO 9999',
 'RO DE CHEQUE NUMERO 9999 ',
 'RECIBIDO DE BMONEX POR CONCEPTO ORDEN DE XXXXX XXXXXXXXXXXXXX XXXXXXX CTA.ORDENANTE 000000000000000000 REF.0000000 TRASPASO A CUENTA PROPIA RASTREO: 00000000',
 'RO DE CHEQUE NUMERO 9999 ',
 'RECIBIDO DE BMONEX POR ORDEN DE XXXXXXX XXXXXXXXXXXXXX XXXXXX CTA.ORDENANTE 000000000000000000 REF.0000000 TRASPASO A CUENTA PROPIA RASTREO: 00000000 ',
 'DICION ORDEN AL EXTRANJERO 0000000 O.P. ENVIADA AL EXT. ',
 'DEL CUENTE 0000000 O.P. ENVIADA AL EXT. ',
 'DEL CLIENTE 0000000 O.P. ENVIADA AL EXT. ',
 'RECIBIDO DE BMONEX POR ORDEN DE XXXXXXXX XXXXXXXXXXXXXX XXXXXX CTA.ORDENANTE 000000000000000000 REF.0000000 TRASPASO A CUENTA PROPIA RASTREO: 00000000 30 SEP',
 'RO DE CHEQUE NUMERO 0000 30 SEP ',
 'QUES EXENTOS

In [70]:
conceptos1_3=[]
for i in conceptos1_2:
    if i.startswith('ANTERIOR'):
        conceptos1_3.append('SALDO '+i)
    elif i.startswith('RECIBIDO'):
        conceptos1_3.append('PAGO '+i)
    elif i.startswith('RO'):
        conceptos1_3.append('COB'+i)
        
    elif i.startswith('DEL'):
        conceptos1_3.append('POR INSTRUCCION '+i)
    
    elif i.startswith('DICION'):
        conceptos1_3.append('EXPE'+i)
    elif i.startswith('QUES'):
        conceptos1_3.append('NUMERO DE CHE'+i)
        

In [73]:
conceptos1_3

['SALDO ANTERIOR ',
 'PAGO RECIBIDO DE BMONEX POR ORDEN DE NOMBRE XXXXXXXXXXX CTA.ORDENANTE 000000000000000000 REF.0000000 TRASPASO A CUENTA PROPIA RASTREO: 00000000',
 'COBRO DE CHEQUE NUMERO 9999',
 'COBRO DE CHEQUE NUMERO 9999',
 'COBRO DE CHEQUE NUMERO 9999',
 'COBRO DE CHEQUE NUMERO 9999 ',
 'PAGO RECIBIDO DE BMONEX POR CONCEPTO ORDEN DE XXXXX XXXXXXXXXXXXXX XXXXXXX CTA.ORDENANTE 000000000000000000 REF.0000000 TRASPASO A CUENTA PROPIA RASTREO: 00000000',
 'COBRO DE CHEQUE NUMERO 9999 ',
 'PAGO RECIBIDO DE BMONEX POR ORDEN DE XXXXXXX XXXXXXXXXXXXXX XXXXXX CTA.ORDENANTE 000000000000000000 REF.0000000 TRASPASO A CUENTA PROPIA RASTREO: 00000000 ',
 'EXPEDICION ORDEN AL EXTRANJERO 0000000 O.P. ENVIADA AL EXT. ',
 'POR INSTRUCCION DEL CUENTE 0000000 O.P. ENVIADA AL EXT. ',
 'POR INSTRUCCION DEL CLIENTE 0000000 O.P. ENVIADA AL EXT. ',
 'PAGO RECIBIDO DE BMONEX POR ORDEN DE XXXXXXXX XXXXXXXXXXXXXX XXXXXX CTA.ORDENANTE 000000000000000000 REF.0000000 TRASPASO A CUENTA PROPIA RASTREO: 000000

# USando libreria tabula

In [21]:
#pip install tabula-py

In [22]:
import tabula

In [23]:
tabu=tabula.read_pdf(archivo, pages='all')


In [138]:
#saldos
saldos_1=[]
for i in range(len(tabu)):
    tabu[i].fillna(0,inplace=True)
    try:
        for j in tabu[i]['SALDO']:
            if j!=0:
                saldos_1.append(j)
    except:
        pass

In [141]:
saldos_2=[]
for i in saldos_1:
    if re.search(r'\d+\,\d+\.\d{2}', i):
        saldos_2.append(i)

In [146]:
#ordeno saldos
saldos_2

['9,021.42',
 '9,021.42',
 '9,021.42',
 '9,021.42',
 '8,021.42',
 '9,021.42',
 '9,021.42',
 '9,021.42']

In [142]:
#depositos
depositos_1=[]
for i in range(len(tabu)):
    tabu[i].fillna(0,inplace=True)
    try:
        for j in tabu[i]['DEPOSITOS']:
            if j!=0:
                depositos_1.append(j)
    except:
        pass

In [145]:
depositos_2=[]
for i in depositos_1:
    if re.search(r'\d+\,\d+\.\d{2}', i):
        depositos_2.append(i)
depositos_2

['5,000.00', '9,000.00', '1,984.00', '5,000.00']

In [179]:
#organizo depositos
depositos_3=[]
contador=0
for i in conceptos1_3:
    if i.startswith('PAGO'):
        depositos_3.append(depositos_2[contador])
        contador=contador+1
    else:
        depositos_3.append(0)

In [185]:
print(depositos_3)

[0, '5,000.00', 0, 0, 0, 0, '9,000.00', 0, '1,984.00', 0, 0, 0, '5,000.00', 0, 0]


In [191]:
len(depositos_3)

15

In [160]:
#retiros
retiros_1=[]
for i in range(len(tabu)):
    tabu[i].fillna(0,inplace=True)
    try:
        for j in tabu[i]['RETIROS']:
            if j!=0:
                retiros_1.append(j)
    except:
        try:
            for j in tabu[i]['CONCEPTO ORDEN RETIROS']:
                if j!=0:
                    k= re.findall(r'\d+\,\d+\.\d{2}$',j)
                    retiros_1.append(k)
        except:
            pass

In [214]:
#limpio
retiros_2=[]
for i in retiros_1:
    j=re.findall(r'\d+\,\d+\.\d{2}', str(i))
    retiros_2.append(j)

In [215]:
#aplano
retiros_3=[]
for sublist in retiros_2:
    for item in sublist:
        if re.search(r'\d+\,\d+\.\d+',item):
            retiros_3.append(item)

In [216]:
retiros_3

['3,000.00',
 '3,000.00',
 '3,000.00',
 '3,000.00',
 '9,000.00',
 '2,100.00',
 '9,284.00',
 '2,600.00',
 '3,000.00']

In [217]:
#organizo
retiros_4=[]
contador=0
for i in conceptos1_3:
    if i.startswith('COBRO'):
        retiros_4.append(retiros_3[contador])
        contador=contador+1
    elif i.startswith('EXPEDICION'):
        retiros_4.append(retiros_3[contador])
        contador=contador+1
    elif i.startswith('POR'):
        retiros_4.append(retiros_3[contador])
        contador=contador+1
    else:
        retiros_4.append(0)

In [218]:
print(retiros_4)
print(depositos_3)

[0, 0, '3,000.00', '3,000.00', '3,000.00', '3,000.00', 0, '9,000.00', 0, '2,100.00', '9,284.00', '2,600.00', 0, '3,000.00', 0]
[0, '5,000.00', 0, 0, 0, 0, '9,000.00', 0, '1,984.00', 0, 0, 0, '5,000.00', 0, 0]


In [265]:
saldos_3=[saldos_2[0]]
contador=0
for i in range(0,len(depositos_3)):
    if depositos_3[i]==0:
        try:
            saldos_3.append(saldos_2[contador])
            contador=contador+1
        except:
            pass
    else:
        saldos_3.append(0)
saldos_3.append(saldos_2[-1])
saldos_3.append(saldos_2[-1])

In [266]:
len(saldos_3)

15

# junto

In [270]:
estado=pd.DataFrame(list(zip(fechas_pag_1, conceptos1_3, retiros_4, depositos_3, saldos_3)),
              columns=['fecha','concepto','retiro', 'deposito','saldo'])

In [272]:
estado['banco']='banamex'

In [273]:
estado['nacionalidad']='nacional'

In [274]:
estado['fecha de corte']=periodo

In [275]:
estado['titular']=nombre

In [276]:
estado['cuenta']=cliente

In [277]:
estado['saldo inicial']= estado['saldo'][0]

In [278]:
if re.search('usd',text_str.lower()) or re.search('dolar', text_str.lower()) or re.search('dollar', text_str.lower()):
    moneda='usd'
else:
    moneda='mxn'

In [279]:
moneda

'mxn'

In [280]:
estado['divisa']=moneda

In [281]:
edo_cuenta= estado[['banco','nacionalidad','fecha de corte', 'titular','cuenta','fecha','concepto','saldo inicial',
                    'retiro','deposito','saldo','divisa']]

In [285]:
edo_cuenta

,banco,nacionalidad,fecha de corte,titular,cuenta,fecha,concepto,saldo inicial,retiro,deposito,saldo,divisa
0,banamex,nacional,RESUMEN DEL: 01/SEP/2020 AL 30/SEP/2020,NOMBRE APELLIDO P APELLIDO M,cliente 9999999,0,SALDO ANTERIOR,"9,021.42",0,0,"9,021.42",mxn
1,banamex,nacional,RESUMEN DEL: 01/SEP/2020 AL 30/SEP/2020,NOMBRE APELLIDO P APELLIDO M,cliente 9999999,01 SEP,PAGO RECIBIDO DE BMONEX POR ORDEN DE NOMBRE XX...,"9,021.42",0,"5,000.00","9,021.42",mxn
2,banamex,nacional,RESUMEN DEL: 01/SEP/2020 AL 30/SEP/2020,NOMBRE APELLIDO P APELLIDO M,cliente 9999999,01 SEP,COBRO DE CHEQUE NUMERO 9999,"9,021.42","3,000.00",0,0,mxn
3,banamex,nacional,RESUMEN DEL: 01/SEP/2020 AL 30/SEP/2020,NOMBRE APELLIDO P APELLIDO M,cliente 9999999,02 SEP,COBRO DE CHEQUE NUMERO 9999,"9,021.42","3,000.00",0,"9,021.42",mxn
4,banamex,nacional,RESUMEN DEL: 01/SEP/2020 AL 30/SEP/2020,NOMBRE APELLIDO P APELLIDO M,cliente 9999999,10 SEP,COBRO DE CHEQUE NUMERO 9999,"9,021.42","3,000.00",0,"9,021.42",mxn
5,banamex,nacional,RESUMEN DEL: 01/SEP/2020 AL 30/SEP/2020,NOMBRE APELLIDO P APELLIDO M,cliente 9999999,17 SEP,COBRO DE CHEQUE NUMERO 9999,"9,021.42","3,000.00",0,"9,021.42",mxn
6,banamex,nacional,RESUMEN DEL: 01/SEP/2020 AL 30/SEP/2020,NOMBRE APELLIDO P APELLIDO M,cliente 9999999,17 SEP,PAGO RECIBIDO DE BMONEX POR CONCEPTO ORDEN DE ...,"9,021.42",0,"9,000.00","8,021.42",mxn
7,banamex,nacional,RESUMEN DEL: 01/SEP/2020 AL 30/SEP/2020,NOMBRE APELLIDO P APELLIDO M,cliente 9999999,18 SEP,COBRO DE CHEQUE NUMERO 9999,"9,021.42","9,000.00",0,0,mxn
8,banamex,nacional,RESUMEN DEL: 01/SEP/2020 AL 30/SEP/2020,NOMBRE APELLIDO P APELLIDO M,cliente 9999999,24 SEP,PAGO RECIBIDO DE BMONEX POR ORDEN DE XXXXXXX X...,"9,021.42",0,"1,984.00","9,021.42",mxn
9,banamex,nacional,RESUMEN DEL: 01/SEP/2020 AL 30/SEP/2020,NOMBRE APELLIDO P APELLIDO M,cliente 9999999,24 SEP,EXPEDICION ORDEN AL EXTRANJERO 0000000 O.P. EN...,"9,021.42","2,100.00",0,0,mxn


In [283]:
nombre_archivo=re.findall(r'\d+/\w+/\d+',periodo)[0]+'-banamex-cuenta-cheques-banamex'
nombre_archivo=nombre_archivo.replace('/','-')
nombre_archivo

'01-SEP-2020-banamex-cuenta-cheques-banamex'

In [284]:
#exporto
#edo_cuenta.to_csv('tresalia\Bancos\Banamex_nacional\salida\\'+nombre_archivo+'.txt' , sep='\t', index=False)